In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import seaborn as sns
sns.set()


In [3]:
df = pd.read_csv('stock fundamental data.csv')
df.head()


,gvkey,tic,fyear,indfmt,consol,popsrc,datafmt,curcd,act,ap,...,spi,txp,txt,xad,xido,xint,xrd,xsga,costat,mkvalt
0,1004,AIR,2009.0,INDL,C,D,STD,USD,863.429,114.906,...,-4.302,3.263,20.986,NaN,0.000,26.832,NaN,151.904,A,777.8348
1,1004,AIR,2010.0,INDL,C,D,STD,USD,913.985,185.096,...,-1.536,0.000,35.364,NaN,-3.313,30.670,NaN,171.399,A,1049.8206
2,1004,AIR,2011.0,INDL,C,D,STD,USD,1063.272,201.405,...,-13.864,0.000,25.480,NaN,0.000,37.772,NaN,189.397,A,485.2897
3,1004,AIR,2012.0,INDL,C,D,STD,USD,1033.700,149.300,...,-21.100,0.000,26.700,NaN,0.000,41.600,NaN,207.400,A,790.0029
4,1004,AIR,2013.0,INDL,C,D,STD,USD,1116.900,171.100,...,0.000,0.000,32.100,NaN,0.000,42.000,NaN,197.600,A,961.3080


# Start preprocessing data


In [4]:
#create column 'future mkvalt' that holds the market value of the next year
condition1 = df['tic'].shift(-1) == df['tic']
condition2 = df['fyear'].shift(-1) == df['fyear'] + 1
df['future_mkvalt'] = np.where(condition1 & condition2, df['mkvalt'].shift(-1), np.nan)

#drop nulls
df = df.dropna(subset=['mkvalt', 'future_mkvalt'], how='all')

#percent change is the percent change in market value from one year to the next
df['percent_change'] = (df['future_mkvalt']-df['mkvalt'])/df['mkvalt']

#if percent change is greater than 20%, classify as undervalued
df['undervalued'] = (df['percent_change'] > 0.10)

In [5]:
#fill null expenses with 0
df['xrd'] = df['xad'].fillna(0)
df['xad'] = df['xad'].fillna(0)
df['xsga'] = df['xsga'].fillna(0)
df['xint'] = df['xsga'].fillna(0)

#combine all expenses columns into one column
expenses_list = ['xrd', 'xad', 'xsga', 'xint', 'nopio', 'cogs']
df['expenses'] = df[expenses_list].sum(axis=1)

df = df.drop(columns=expenses_list)

#delete duplicate rows due to formatting
condition = df['indfmt'] != "FS"
df = df[condition]

#drop unneccessary columns
df = df.drop(columns=['indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'gvkey', 'costat', 'dlcch'])

#remove rows with null values
condition = df.isnull().sum(axis=1) < 1
df = df[condition]


In [6]:
df.head(5)

,tic,fyear,act,ap,at,ceq,che,dlc,dltt,dp,...,sale,spi,txp,txt,xido,mkvalt,future_mkvalt,percent_change,undervalued,expenses
0,AIR,2009.0,863.429,114.906,1501.042,746.906,79.370,100.833,336.191,38.930,...,1352.151,-4.302,3.263,20.986,0.000,777.8348,1049.8206,0.349670,True,1368.672
1,AIR,2010.0,913.985,185.096,1703.727,835.845,57.433,114.075,329.802,59.296,...,1775.782,-1.536,0.000,35.364,-3.313,1049.8206,485.2897,-0.537740,False,1754.213
2,AIR,2011.0,1063.272,201.405,2195.653,864.649,67.720,122.865,669.489,80.333,...,2074.498,-13.864,0.000,25.480,0.000,485.2897,790.0029,0.627900,True,2042.744
3,AIR,2012.0,1033.700,149.300,2136.900,918.600,75.300,86.400,622.200,108.600,...,2167.100,-21.100,0.000,26.700,0.000,790.0029,961.3080,0.216841,True,2136.100
4,AIR,2013.0,1116.900,171.100,2199.500,999.500,89.200,69.700,564.300,113.400,...,2035.000,0.000,0.000,32.100,0.000,961.3080,1046.3954,0.088512,False,1979.700


In [7]:
df.isnull().sum()

tic               0
fyear             0
act               0
ap                0
at                0
ceq               0
che               0
dlc               0
dltt              0
dp                0
dvc               0
ib                0
intan             0
invt              0
ivao              0
lct               0
lt                0
ppent             0
rect              0
sale              0
spi               0
txp               0
txt               0
xido              0
mkvalt            0
future_mkvalt     0
percent_change    0
undervalued       0
expenses          0
dtype: int64

In [8]:
df.describe()

,fyear,act,ap,at,ceq,che,dlc,dltt,dp,dvc,...,rect,sale,spi,txp,txt,xido,mkvalt,future_mkvalt,percent_change,expenses
count,18538.000000,18538.000000,18538.000000,18538.000000,18538.000000,18538.000000,18538.000000,18538.000000,18538.000000,18538.000000,...,18538.000000,18538.000000,18538.000000,18538.000000,18538.000000,18538.000000,18538.000000,1.853800e+04,18538.000000,18538.000000
mean,2014.252616,1602.344479,354.461054,5369.508227,1974.783587,606.672774,171.597051,1450.590908,221.991089,124.402125,...,487.509058,4009.085775,-45.192709,25.373918,101.652333,5.324156,6371.865753,7.015074e+03,0.284440,4099.828824
std,2.583827,5862.419764,1524.118420,17397.819838,7245.189089,3428.585543,971.136028,4851.029901,865.581610,595.802130,...,1750.696922,15336.614821,405.900904,157.560783,632.142553,186.590999,25178.458659,2.867753e+04,2.612639,16275.395525
min,2009.000000,0.000000,0.000000,0.000000,-9660.000000,0.000000,0.000000,0.000000,0.000000,-0.029000,...,0.000000,0.000000,-10753.000000,-0.249000,-9956.000000,-4893.000000,0.236200,2.362000e-01,-0.993263,-3111.710000
25%,2012.000000,75.472250,4.729500,150.318250,63.753750,20.490000,0.000000,0.000000,3.601750,0.000000,...,9.978000,87.236000,-15.453000,0.000000,0.000000,0.000000,195.903025,2.004954e+02,-0.174187,120.376250
50%,2014.000000,280.602000,29.265000,724.749500,297.741000,83.665500,2.300000,78.000000,25.285000,0.000000,...,68.524000,565.198000,-0.933500,0.000000,4.599500,0.000000,875.441250,9.499224e+02,0.074305,616.934000
75%,2016.000000,1006.711750,156.056250,3109.408750,1166.100250,299.015750,31.147250,837.863000,117.896750,25.032500,...,305.601250,2443.360500,0.000000,4.350000,45.598000,0.000000,3506.577175,3.868312e+03,0.365649,2533.854750
max,2018.000000,169662.000000,38487.000000,274098.000000,177628.000000,133768.000000,53493.000000,113642.000000,21037.000000,12917.000000,...,65812.000000,483521.000000,22056.000000,5084.000000,20626.000000,15914.500000,757028.970000,1.023856e+06,157.373350,552567.000000


# Rename the columns to names with more meaning

In [46]:
#load data dictionary 
data_dict = pd.read_csv("data dictionary.csv", index_col= False)
data_dict.head()

,Mnemonic,Description,Unnamed: 2,Unnamed: 3,Category,Unnamed: 5,Units
0,act,Total Current Assets,NaN,NaN,balance sheet,NaN,millions
1,ap,Accounts Payable,NaN,NaN,balance sheet,NaN,millions
2,at,Total Assets,NaN,NaN,balance sheet,NaN,millions
3,ceq,Total Common/Ordinary Equity,NaN,NaN,balance sheet,NaN,millions
4,che,Cash and Short-Term Investments,NaN,NaN,balance sheet,NaN,millions


In [47]:
# drop all columns except Mnemonic and description
data_dict.drop( columns=['Unnamed: 2','Unnamed: 3','Category','Unnamed: 5','Units'], inplace=True)
data_dict.head()

,Mnemonic,Description
0,act,Total Current Assets
1,ap,Accounts Payable
2,at,Total Assets
3,ceq,Total Common/Ordinary Equity
4,che,Cash and Short-Term Investments


In [48]:
# possibly use this data frame to rename columns in an easier way

# Data Visualization

In [10]:
# where to start???